In [ ]:
%pip install -U pandas numpy scikit-learn matplotlib openpyxl scikeras

In [ ]:
import sys, tensorflow as tf
print(sys.executable) 
print("TF:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))

In [ ]:

import os, random, numpy as np, pandas as pd, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers


random.seed(42); np.random.seed(42); tf.random.set_seed(42)


os.makedirs("outputs", exist_ok=True)

print("TensorFlow:", tf.__version__)

In [ ]:
TARGET = "Grid  Power"  
FILTER_Y_EQ_ZERO = True   


raw_data = pd.read_excel('raw data.xlsx')


raw_data = raw_data.drop(columns=['times'], errors='ignore').copy()


if FILTER_Y_EQ_ZERO:
    raw_data = raw_data[raw_data[TARGET] != 0].copy()


num_cols = [c for c in raw_data.columns if c != TARGET]
raw_data[num_cols] = raw_data[num_cols].apply(pd.to_numeric, errors='coerce')
raw_data = raw_data.dropna().reset_index(drop=True)

print("Data shape:", raw_data.shape)
display(raw_data.head(3))


In [ ]:

# Train/Test (80/20)
train_dataset = raw_data.sample(frac=0.8, random_state=0)
test_dataset  = raw_data.drop(train_dataset.index)


test_x_data = test_dataset.drop(columns=[TARGET]).copy()
test_y_data = test_dataset[TARGET].copy()


labeled_train_data   = train_dataset.sample(frac=0.6, random_state=0).copy()
unlabeled_train_data = train_dataset.drop(labeled_train_data.index).copy()


unlabeled_train_data_actual = unlabeled_train_data.pop(TARGET).copy()
labeled_data_labels         = labeled_train_data.pop(TARGET).copy()

len(train_dataset), len(labeled_train_data), len(unlabeled_train_data), len(test_dataset)


In [ ]:


from sklearn.preprocessing import StandardScaler


print("Columns:", train_dataset.columns.tolist())


feature_cols = [c for c in train_dataset.columns if c != TARGET]


scaler_fs = StandardScaler().fit(labeled_train_data[feature_cols])


X_fs_tr = scaler_fs.transform(labeled_train_data[feature_cols]).astype("float32")
y_fs_tr = labeled_data_labels.values.astype("float32")  

X_fs_te = scaler_fs.transform(test_dataset[feature_cols]).astype("float32")
y_fs_te = test_dataset[TARGET].values.astype("float32")

INPUT_DIM_FS = X_fs_tr.shape[1]
print("Supervised shapes → X_tr:", X_fs_tr.shape, " X_te:", X_fs_te.shape, "  INPUT_DIM_FS:", INPUT_DIM_FS)


In [ ]:
# === Mean-Teacher (Π-model, regression) — data prep ===


X_u = scaler_fs.transform(unlabeled_train_data[feature_cols]).astype("float32")


n_all = X_fs_tr.shape[0]
n_val = int(round(n_all * 0.20))
X_l_tr, y_l_tr = X_fs_tr[:n_all - n_val], y_fs_tr[:n_all - n_val]
X_l_va, y_l_va = X_fs_tr[n_all - n_val:], y_fs_tr[n_all - n_val:]

X_u.shape, X_l_tr.shape, X_l_va.shape

In [ ]:

def build_model(
    input_dim: int,
    layers_: int = 3,
    units: int = 32,
    dropout: float = 0.0,
    l2: float = 0.0,
    activation: str = "relu",      
    learning_rate: float = 1e-3,
    loss: str = "mae",
    metrics=("mae","mse"),
):
    inputs = keras.Input(shape=(input_dim,))
    x = inputs
    kreg = regularizers.l2(l2) if (l2 and l2 > 0) else None
    for _ in range(layers_):
        x = layers.Dense(units, kernel_regularizer=kreg)(x)
        if activation == "leaky_relu":
            x = layers.LeakyReLU(alpha=0.2)(x)
        else:
            x = layers.Activation(activation)(x)
        if dropout and dropout > 0:
            x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=list(metrics),
    )
    return model

In [ ]:
# === Mean-Teacher (Π-model, regression) — train & evaluate & save ===

from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error, mean_squared_error


EPOCHS         = 120
BATCH_L        = 64
BATCH_U        = 64
LR             = 3e-4     
EMA_M          = 0.999    
LAMBDA_U_MAX   = 0.2      
RAMP_UP_EPO    = 30        
BURN_IN_EPO    = 8         
NOISE_STD      = 0.02      
CLIP_NORM      = 1.0      


X_l_tr = X_l_tr.astype("float32"); y_l_tr = y_l_tr.astype("float32")
X_l_va = X_l_va.astype("float32"); y_l_va = y_l_va.astype("float32")
X_fs_te = X_fs_te.astype("float32"); y_fs_te = y_fs_te.astype("float32")
X_u = X_u.astype("float32")


In [ ]:

student = build_model(INPUT_DIM_FS, layers_=3, units=32, dropout=0.0, l2=0.0,
                      activation="relu", learning_rate=LR, loss="mae")
pre_callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_mae", patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_mae", factor=0.5, patience=5, min_lr=1e-5),
]
student.fit(
    X_l_tr, y_l_tr,
    validation_data=(X_l_va, y_l_va),
    epochs=40, batch_size=BATCH_L, verbose=0,
    callbacks=pre_callbacks
)


teacher = build_model(INPUT_DIM_FS, layers_=3, units=32, dropout=0.0, l2=0.0,
                      activation="relu", learning_rate=LR, loss="mae")
teacher.set_weights(student.get_weights())

optimizer = keras.optimizers.Adam(learning_rate=LR, clipnorm=CLIP_NORM)

def ds_labeled(X, y, bs): return tf.data.Dataset.from_tensor_slices((X, y)).shuffle(2048).repeat().batch(bs)
def ds_unlabeled(X, bs):  return tf.data.Dataset.from_tensor_slices(X).shuffle(2048).repeat().batch(bs)
dsL = iter(ds_labeled(X_l_tr, y_l_tr, BATCH_L))
dsU = iter(ds_unlabeled(X_u, BATCH_U))
steps_per_epoch = max(len(X_l_tr) // BATCH_L, 1)

@tf.function
def add_noise(x, std=NOISE_STD):
    return x + tf.random.normal(tf.shape(x), stddev=std)

train_hist = []
best_val_mae = float("inf")
best_teacher_weights = None

for epoch in range(1, EPOCHS + 1):
    # ramp-up & burn-in
    if epoch <= BURN_IN_EPO:
        lambda_u = 0.0
    else:
        ramp = tf.clip_by_value((epoch - BURN_IN_EPO) / max(RAMP_UP_EPO, 1), 0.0, 1.0)
        lambda_u = float(LAMBDA_U_MAX * ramp.numpy())

    sup_mae_acc = keras.metrics.Mean()
    cons_mse_acc = keras.metrics.Mean()
    total_acc = keras.metrics.Mean()

    for step in range(steps_per_epoch):
        xb_l, yb_l = next(dsL)
        xb_u = next(dsU)

        with tf.GradientTape() as tape:
        
            pred_l = tf.squeeze(student(add_noise(xb_l, NOISE_STD), training=True), axis=-1)
            sup_loss = tf.reduce_mean(tf.abs(yb_l - pred_l))  # L1/MAE

            u1 = add_noise(xb_u, NOISE_STD)
            u2 = add_noise(xb_u, NOISE_STD)
            stu_u = tf.squeeze(student(u1, training=True), axis=-1)
            tea_u = tf.squeeze(teacher(u2, training=False), axis=-1)
            cons_loss = tf.reduce_mean(tf.square(stu_u - tf.stop_gradient(tea_u)))  # L2/MSE

            loss = sup_loss + lambda_u * cons_loss

        grads = tape.gradient(loss, student.trainable_variables)
        optimizer.apply_gradients(zip(grads, student.trainable_variables))

     
        tw, sw = teacher.get_weights(), student.get_weights()
        teacher.set_weights([EMA_M * t + (1.0 - EMA_M) * s for t, s in zip(tw, sw)])

        sup_mae_acc.update_state(sup_loss)
        cons_mse_acc.update_state(cons_loss)
        total_acc.update_state(loss)

  
    val_pred = teacher.predict(X_l_va, verbose=0).ravel()
    val_mae = float(np.mean(np.abs(val_pred - y_l_va)))

    train_hist.append({
        "epoch": epoch,
        "sup_mae": float(sup_mae_acc.result().numpy()),
        "cons_mse": float(cons_mse_acc.result().numpy()),
        "total": float(total_acc.result().numpy()),
        "val_mae": val_mae,
        "lambda_u": lambda_u,
    })

  
    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_teacher_weights = teacher.get_weights()


if best_teacher_weights is not None:
    teacher.set_weights(best_teacher_weights)


In [ ]:

def metrics_block(y_true, y_pred):
    mae  = mean_absolute_error(y_true, y_pred)
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2   = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return {"Loss": mae, "MAE": mae, "MSE": mse, "RMSE": rmse, "R2": r2, "MAPE": mape}

yhat_tr = teacher.predict(X_l_tr, verbose=0).ravel()
yhat_va = teacher.predict(X_l_va, verbose=0).ravel()
yhat_te = teacher.predict(X_fs_te, verbose=0).ravel()

m_tr = metrics_block(y_l_tr, yhat_tr)
m_va = metrics_block(y_l_va, yhat_va)
m_te = metrics_block(y_fs_te, yhat_te)

hist_df = pd.DataFrame(train_hist)
pred_train_df = pd.DataFrame({"y_true": y_l_tr, "y_pred": yhat_tr, "residual": y_l_tr - yhat_tr})
pred_val_df   = pd.DataFrame({"y_true": y_l_va, "y_pred": yhat_va, "residual": y_l_va - yhat_va})
pred_test_df  = pd.DataFrame({"y_true": y_fs_te, "y_pred": yhat_te, "residual": y_fs_te - yhat_te})
summary_df = pd.DataFrame.from_dict({"Train": m_tr, "Val": m_va, "Test": m_te}, orient="index")[["Loss","MAE","MSE","RMSE","R2","MAPE"]]

os.makedirs("outputs", exist_ok=True)
pd.DataFrame({"y_true": y_fs_te, "y_pred": yhat_te}).to_excel("outputs/mean_teacher_test_preds.xlsx", index=False)
with pd.ExcelWriter("outputs/mean_teacher_errors.xlsx", engine="openpyxl") as w:
    hist_df.to_excel(w, sheet_name="history", index=False)
    summary_df.to_excel(w, sheet_name="summary")
    pred_train_df.to_excel(w, sheet_name="pred_train", index=False)
    pred_val_df.to_excel(w,   sheet_name="pred_val",   index=False)
    pred_test_df.to_excel(w,  sheet_name="pred_test",  index=False)

print("[Mean-Teacher] Summary:\n", summary_df)
